In [3]:
import numpy as nnp
import matplotlib.pyplot as plt

from lr_utils import *

In [4]:
# 고양이인 데이터와 고양이가 아닌 데이터를
# 로지스틱 회귀를 통해 구분해보기
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [ ]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T

In [ ]:
#utils -param 시작, sigmoid

def initialize_with_zeros(dim):
    #zeros, ones는 zeros((차원1, 차원2))과 같은식으로 사용해야 함
    w = np.zeros((dim,1))
    b = 0.
    return w, b

def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [17]:
def propagate(w,b,X,Y):
    m = X.shape[1]
    Z = w.T@X + b
    A = sigmoid(Z)

    cost = (Y * np.log(A) + (1-Y) * np.log(1-A)).sum() / -m

    dw = X @ (A-Y).T / m
    db = (A-Y).sum() / m
    grads = {
        "dw" : dw,
        "db" : db,
    }
    return grads, cost

In [20]:
def optimize(w, b, X, Y, num_iterations = 2000, lr = 0.009, print_cost = True):
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w,b,X,Y)
        dw = grads["dw"]
        db = grads["db"]
        #update param
        #deepcopy를 사용하지 않는다면 optimize 외부에서 넘어온 w와 b의 값이
        #어딘가에서 변할 수도 있음
        w = w- lr * dw
        b = b- lr * db
        
        if (i+1) % 100 ==0:
            costs.append(cost)

            if print_cost:
                print(f"Cost After {i+1} iterations : {cost}")
    params = {
        "w" : w, 
        "b" : b}
    
    grads = {"dw" : dw,
             "db" : db,}
    return params, grads, costs
    


In [21]:
train_set_x = train_set_x_flatten / train_set_x_flatten.max()
test_set_x = test_set_x_flatten / train_set_x_flatten.max() # 흠 스케일링을 똑같이 해야겠죠?
w,b = initialize_with_zeros(train_set_x.shape[0])
params, grads, costs = optimize(w,b,train_set_x,train_set_y, )

Cost After 100 iterations : 0.47715649504824453
Cost After 200 iterations : 0.8471092191546921
Cost After 300 iterations : 0.5837524449397078
Cost After 400 iterations : 0.48113380121374033
Cost After 500 iterations : 0.36121067514479355
Cost After 600 iterations : 0.24121803161715624
Cost After 700 iterations : 0.16856909780439225
Cost After 800 iterations : 0.15051961104077807
Cost After 900 iterations : 0.13960214924932912
Cost After 1000 iterations : 0.1303981662836805
Cost After 1100 iterations : 0.1223950112584566
Cost After 1200 iterations : 0.11532321915155781
Cost After 1300 iterations : 0.10901077034429985
Cost After 1400 iterations : 0.10333458691721242
Cost After 1500 iterations : 0.0982005870101832
Cost After 1600 iterations : 0.09353397980325093
Cost After 1700 iterations : 0.08927382998229712
Cost After 1800 iterations : 0.08536965873672563
Cost After 1900 iterations : 0.0817791391225479
Cost After 2000 iterations : 0.07846644198882445
